In [1]:
print('hi')

hi


In [8]:
%pip install crewai crewai_tools

  Using cached crewai_tools-0.40.1-py3-none-any.whl.metadata (5.5 kB)
  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached embedchain-0.1.128-py3-none-any.whl.metadata (9.2 kB)
  Using cached lancedb-0.21.2-cp39-abi3-win_amd64.whl.metadata (4.2 kB)
  Using cached pyright-1.1.399-py3-none-any.whl.metadata (6.6 kB)
  Using cached pytube-15.0.0-py3-none-any.whl.metadata (5.0 kB)
  Using cached alembic-1.15.2-py3-none-any.whl.metadata (7.3 kB)
  Using cached beautifulsoup4-4.13.4-py3-none-any.whl.metadata (3.8 kB)
  Using cached chromadb-0.5.23-py3-none-any.whl.metadata (6.8 kB)
  Using cached gptcache-0.1.44-py3-none-any.whl.metadata (24 kB)
  Using cached langchain_cohere-0.3.5-py3-none-any.whl.metadata (6.7 kB)
  Using cached langchain_openai-0.2.14-py3-none-any.whl.metadata (2.7 kB)
  Using cached langsmith-0.3.33-py3-none-any.whl.metadata (15 kB)
  Using cached mem0ai-0.1.93-py3-none-any.whl.metadata (8.6 kB)
  Using cached pypdf-5.4.0-py3-none-any.whl.metadat

  You can safely remove it manually.


In [2]:
from crewai import Agent, Task, Crew, LLM
import os
import yaml 
from dotenv import load_dotenv

In [3]:
from dotenv import load_dotenv
load_dotenv(dotenv_path='../.env',override=True)
api_key=os.getenv("GROQ_API_KEY")
os.environ["GROQ_API_KEY"]=api_key
llm_saler = LLM(model="groq/llama3.1 8B")

In [4]:
# Define file paths for YAML configurations
files = {
    'lead_agents': 'config/lead_qualification_agents.yaml',
    'lead_tasks': 'config/lead_qualification_tasks.yaml',
    'email_agents': 'config/email_engagement_agents.yaml',
    'email_tasks': 'config/email_engagement_tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
lead_agents_config = configs['lead_agents']
lead_tasks_config = configs['lead_tasks']
email_agents_config = configs['email_agents']
email_tasks_config = configs['email_tasks']

In [6]:
from pydantic import BaseModel, Field
from typing import Dict, Optional, List, Set, Tuple

class LeadPersonalInfo(BaseModel):
    name: str = Field(..., description="The full name of the lead.")
    job_title: str = Field(..., description="The job title of the lead.")
    role_relevance: int = Field(..., ge=0, le=10, description="A score representing how relevant the lead's role is to the decision-making process (0-10).")
    professional_background: Optional[str] = Field(..., description="A brief description of the lead's professional background.")

class CompanyInfo(BaseModel):
    company_name: str = Field(..., description="The name of the company the lead works for.")
    industry: str = Field(..., description="The industry in which the company operates.")
    company_size: int = Field(..., description="The size of the company in terms of employee count.")
    revenue: Optional[float] = Field(None, description="The annual revenue of the company, if available.")
    market_presence: int = Field(..., ge=0, le=10, description="A score representing the company's market presence (0-10).")

class LeadScore(BaseModel):
    score: int = Field(..., ge=0, le=100, description="The final score assigned to the lead (0-100).")
    scoring_criteria: List[str] = Field(..., description="The criteria used to determine the lead's score.")
    validation_notes: Optional[str] = Field(None, description="Any notes regarding the validation of the lead score.")

class LeadScoringResult(BaseModel):
    personal_info: LeadPersonalInfo = Field(..., description="Personal information about the lead.")
    company_info: CompanyInfo = Field(..., description="Information about the lead's company.")
    lead_score: LeadScore = Field(..., description="The calculated score and related information for the lead.")


In [9]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

d:\Alex_SoloStudy\Agent\Sales_Pipeline\.venv\Lib\site-packages\pydantic\_internal\_generate_schema.py:623: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


In [10]:
# Creating Agents
lead_data_agent = Agent(
  config=lead_agents_config['lead_data_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()]
)

cultural_fit_agent = Agent(
  config=lead_agents_config['cultural_fit_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()]
)

scoring_validation_agent = Agent(
  config=lead_agents_config['scoring_validation_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()]
)

# Creating Tasks
lead_data_task = Task(
  config=lead_tasks_config['lead_data_collection'],
  agent=lead_data_agent
)

cultural_fit_task = Task(
  config=lead_tasks_config['cultural_fit_analysis'],
  agent=cultural_fit_agent
)

scoring_validation_task = Task(
  config=lead_tasks_config['lead_scoring_and_validation'],
  agent=scoring_validation_agent,
  context=[lead_data_task, cultural_fit_task],
  output_pydantic=LeadScoringResult
)

# Creating Crew
lead_scoring_crew = Crew(
  agents=[
    lead_data_agent,
    cultural_fit_agent,
    scoring_validation_agent
  ],
  tasks=[
    lead_data_task,
    cultural_fit_task,
    scoring_validation_task
  ],
  verbose=True
)